In [25]:
FILE_LIST = "files.p"
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

In [26]:
import regex
word_pattern = "\p{Letter}+"

Oblicz statystykę występowania bigramów słów, pomijając w tekście wszystkie wyrazy, które nie stanowią słów.

In [27]:
import os, json

import pickle
from tqdm import tqdm
from collections import defaultdict

bigrams = defaultdict(lambda: 0)
unigrams = defaultdict(lambda: 0)


def load_data():
    files = pickle.load(open(FILE_LIST, 'rb'))
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [x["textContent"] for x in data["items"] if x["judgmentDate"].startswith(CHOSEN_YEAR)]

            for judgment in judgments:
                previous_word = None

                judgment = regex.sub("<.*?>", "", judgment)
                judgment = regex.sub("-\n(\p{Letter}+)", r"\1", judgment)

                for match in regex.finditer(word_pattern, judgment):
                    [word] = match.captures()

                    if previous_word:
                        bigrams[(previous_word, word.lower())] += 1
                    unigrams[word.lower()] += 1

                    previous_word = word.lower()


load_data()
print(len(bigrams))


100%|██████████| 68/68 [00:36<00:00,  1.87it/s]

2613738


Korzystając z wzoru na punktową informację wzajemną oblicz tę wartość dla wszystkich par słów. Wykorzystaj statystyki unigramów obliczone w poprzednim zadaniu.

In [28]:
import numpy as np

unigram_count = sum(unigrams.values())
bigram_count = sum(bigrams.values())


def get_unigram_probability(unigram):
    return unigrams[unigram] / unigram_count


def get_bigram_probability(bigram):
    return bigrams[bigram] / bigram_count


def get_pmi(bigram):
    x, y = bigram
    return np.log(get_bigram_probability(bigram) / (get_unigram_probability(x) * get_unigram_probability(y)))


In [29]:
bigrams_with_pmi = {
    bigram: get_pmi(bigram) for bigram in bigrams
}

Posortuj bigramy względem malejącej wartości punktowej informacji wzajemnej. Przedstaw 30 pierwszych wyników.



In [30]:
def sort_and_display(bigrams_with_values, top=30):
    sorted_bigrams = sorted(
        bigrams_with_values.items(),
        key=lambda x: x[1],
        reverse=True,
    )

    for bigram, score in sorted_bigrams[:top]:
        print(
            "{} -> {} (count: {}) ".format(
                " ".join(bigram).ljust(35, " "), 
                str(score).ljust(20, " "), 
                bigrams[bigram],
            ),
        )


In [31]:
sorted_bigrams = sort_and_display(bigrams_with_pmi)

domkowi fińskiemu                   -> 16.6181418079068     (count: 1) 
walnymi zgromadzeniami              -> 16.6181418079068     (count: 1) 
wykład inauguracyjny                -> 16.6181418079068     (count: 1) 
reiffeisenlandesbank oberosterreich -> 16.6181418079068     (count: 1) 
iur samuela                         -> 16.6181418079068     (count: 1) 
doprowa dzano                       -> 16.6181418079068     (count: 1) 
twier dzeniom                       -> 16.6181418079068     (count: 1) 
przewle kłość                       -> 16.6181418079068     (count: 1) 
wyni kłej                           -> 16.6181418079068     (count: 1) 
odszko dowania                      -> 16.6181418079068     (count: 1) 
naczyniopochodnymi zawrotami        -> 16.6181418079068     (count: 1) 
zawia domienia                      -> 16.6181418079068     (count: 1) 
pasierbica przysposobiony           -> 16.6181418079068     (count: 1) 
lota lotb                           -> 16.6181418079068     (cou

Korzystając z wzoru na statystykę logarytmiczną opartą o rozkład dwumienny (G2) sporządź analogiczną listę, jak dla punktowej informacji wzajemnej.


In [32]:
import numpy as np


def get_k(bigram):
    bigram_occurrence_count = bigrams[bigram]
    k_11 = bigram_occurrence_count
    k_12 = unigrams[bigram[1]] - bigram_occurrence_count
    k_21 = unigrams[bigram[0]] - bigram_occurrence_count
    k_22 = bigram_count - k_11 - k_12 - k_21

    return np.array(
        [[k_11, k_12],
         [k_21, k_22]]
    )


def H(k):
    """Count Shannon's entropy"""
    N = np.sum(k)

    return np.sum(k / N * np.log(k / N + (k == 0)))


def get_llr(bigram):
    k = get_k(bigram)
    return 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) - H(k.sum(axis=1)))


In [33]:
get_llr(("krwinek", "czerwonych"))

277.5618696509106

In [34]:
get_llr(("domkowi", "fińskiemu"))

35.23465738047877

In [35]:
bigrams_with_llr = {
    bigram: get_llr(bigram) for bigram in tqdm(bigrams)
}

100%|██████████| 2613738/2613738 [02:43<00:00, 16000.16it/s]


In [36]:
sort_and_display(bigrams_with_llr)

art ust                             -> 738378.115228109     (count: 89625) 
z dnia                              -> 480345.94327007694   (count: 77407) 
k p                                 -> 360114.45700380555   (count: 40618) 
art k                               -> 338050.38946833083   (count: 52564) 
sygn akt                            -> 294216.4308285234    (count: 22212) 
nr poz                              -> 287670.8710603976    (count: 27653) 
na podstawie                        -> 273951.5590729033    (count: 34040) 
p c                                 -> 269163.7990452096    (count: 28932) 
dz u                                -> 248984.62267213906   (count: 19293) 
zamówień publicznych                -> 207931.3797994136    (count: 14477) 
ust pkt                             -> 206055.68988247524   (count: 27563) 
zgodnie z                           -> 177200.94076199437   (count: 26381) 
ustawy pzp                          -> 173852.66610368507   (count: 20273) 
w dniu      